In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library')
from hpe_library.lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_vis_frame, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, load_fit3d_one_video, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, get_h36m_limb_lens, part_ids, h36m_part_keypoints, get_h36m_limb_lens
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot

# 주요 디렉토리 경로
user = getpass.getuser()
alphapose_root = f'/home/{user}/codes/AlphaPose'
motionbert_root = f'/home/{user}/codes/MotionBERT'
kookmin_root = f'/home/{user}/Datasets/HAAI/국민대데이터/data'
ap_kookmin_result_root = alphapose_root + "/examples/kookmin_result_5actions"

assert os.path.isdir(alphapose_root), "AlphaPose root directory is not exist"
assert os.path.isdir(motionbert_root), "MotionBERT root directory is not exist"
#assert os.path.isdir(kookmin_root), "Kookmin root directory is not exist"
#assert os.path.isdir(ap_kookmin_result_root), "AlphaPose Kookmin result directory is not exist"

os.chdir(motionbert_root)
from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import get_opts_args
from lib.utils.utils_data import flip_data
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.model.loss import *
from lib.model.training import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_total4, DHDSTformer_total5, DHDSTformer_total6, DHDSTformer_total7, DHDSTformer_total8, \
    DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2
    
dt_file_mapping = {
    'H36M-GT': 'h36m_gt',
    'H36M-GT-CAM_NO_FACTOR': 'h36m_gt',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST': 'h36m_gt_canonical_3d_same_dist',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-S15678_TR_54138969_TS_OTHERS': 'h36m_gt_canonical_3d_same_dist_s15678_tr_54138969_ts_others',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_dist_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z': 'h36m_gt_canonical_3d_same_z',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678_BY_CANON1_6_PRED': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678_BY_CANON2_2_PRED': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'H36M-GT-CAM_NO_FACTOR-S15678_TR_54138969_TS_OTHERS': 'h36m_gt_s15678_tr_54138969_ts_others',
    'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678': 'h36m_gt_tr_s1_ts_s5678',
    'H36M-GT-INPUT_FROM_3D_CANONICAL_SAME_DIST-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_dist_tr_s1_ts_s5678',
    'H36M-GT-TR_S1_TS_S5678': 'h36m_gt_tr_s1_ts_s5678',
    'H36M-GT-WORLD_NO_FACTOR': 'h36m_gt',
    'H36M-SH': 'h36m_sh_conf_cam_source_final',
    'H36M-CANONICALIZATION-GT-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_same_z_tr_s1_ts_s5678',
    'FIT3D-GT-ALL_TEST': 'fit3d_gt_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-ALL_TEST': 'fit3d_gt_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST': 'fit3d_gt_canonical_3d_same_z_all_test',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S03': 'fit3d_gt_canonical_3d_same_z_tr_s03',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710': 'fit3d_gt_canonical_3d_same_z_ts_s4710',
    'FIT3D-GT-CAM_NO_FACTOR-TS_S4710': 'fit3d_gt_ts_s4710',
    'FIT3D-GT-TS_S4710': 'fit3d_gt_ts_s4710',
    '3DHP-GT-CAM_NO_FACTOR-POSEAUG_TEST_2929': 'poseaug_3dhp_test',
    '3DHP-GT-CAM_NO_FACTOR-POSYNDA_TESTSET': '3dhp_gt_test',
    '3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN': '3dhp_gt_test_all_train',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN': '3dhp_gt_canonical_3d_same_z_test_all_train',
    '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_4': '3dhp_gt_test_TS1_4',
    '3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6': '3dhp_gt_test_TS1_6',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_4': '3dhp_gt_test_canonical_3d_from_same_z_TS1_4',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6': '3dhp_gt_test_canonical_3d_from_same_z_TS1_6',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5': 'h36m_gt_canonical_3d_fixed_dist_5',
    'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TR_S1_TS_S5678': 'h36m_gt_canonical_3d_fixed_dist_5_tr_s1_ts_s5678',
    'FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-ALL_TEST': 'fit3d_gt_canonical_3d_fixed_dist_5_all_test',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_TS1_6': '3dhp_gt_test_canonical_3d_from_fixed_dist_5_TS1_6',
    '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_ALL_TRAIN': '3dhp_gt_canonical_3d_fixed_dist_5_test_all_train'
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import requests
from tqdm import tqdm
import copy

# Notion API 키 및 데이터베이스 ID 설정
notion = readpkl('notion.pkl')
NOTION_API_KEY = notion['NOTION_API_KEY']
DATABASE_ID = notion['DATABASE_ID']

# 헤더 설정
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

def get_all_database_pages():
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
    all_pages = []
    has_more = True
    start_cursor = None
    
    while has_more:
        if start_cursor:
            data = {"start_cursor": start_cursor}
        else:
            data = {}
        
        response = requests.post(url, headers=headers, json=data)
        response_data = response.json()
        
        all_pages.extend(response_data["results"])
        has_more = response_data.get("has_more", False)
        start_cursor = response_data.get("next_cursor", None)
    print(f"Total pages: {len(all_pages)}")
    return all_pages

pages = get_all_database_pages()

pages_dict = {}
for page in pages:
    pages_dict[page["id"]] = page
    
model_dict = {}
for page in pages:
    page_id = page["id"]
    property = page["properties"]
    try:
        model_name = property["Model"]['select']['name']
    except: continue
    if model_name not in model_dict:
        model_dict[model_name] = []
    model_dict[model_name].append(page_id)

Total pages: 203


In [3]:
experiment_list_pose3d = [item.split('.')[0] for item in os.listdir('/home/hrai/codes/MotionBERT/configs/pose3d')]
# filtering
blacklist = ['world', 'ft']
mustbe = ['MB', 'gt']
for item in experiment_list_pose3d:
    for keyword in blacklist:
        if keyword in item:
            experiment_list_pose3d.remove(item)
    if item not in experiment_list_pose3d: continue
    for keyword in mustbe:
        if keyword not in item:
            experiment_list_pose3d.remove(item)

In [6]:
# get test list from notion
root = '/home/hrai/codes/MotionBERT/experiments'
if not os.path.isdir(root): os.makedirs(root)
blacklist_keyword = ['PRED', 'POSEAUG', 'posynda', 'TS1_4']

for key in experiment_list_pose3d:
    if key not in model_dict: continue
    if 'dist' in key: 
        if 'fixed_dist_5' in key:
            pass
        else:
            continue
    model_ids = model_dict[key]
    file_path = os.path.join(root, key + '.txt') 
    print(key)
    with open(file_path, 'w') as file:
        for model_id in model_ids:
            page = pages_dict[model_id]
            train = page['properties']['Train dataset']['select'] 
            test = page['properties']['Test dataset']['select']
            # filtering - parent
            if test == None: continue # parent
            
            # get child page's info
            child_page = pages_dict[model_id]
            child_properties = child_page['properties']
            if len(child_properties['Name']['title']) == 0: continue
            child_title = child_properties['Name']['title'][0]['text']['content']
            child_canonical = child_properties['Canonical']['multi_select']
            
            # filtering - keyword
            black = False
            for keyword in blacklist_keyword:
                if keyword in child_title: black = True
            if black: continue
            
            # filtering - input_centering_only_test
            input_centering_only_test = False
            for item in child_canonical:
                if 'only test' in item['name']:
                    input_centering_only_test = True
            if input_centering_only_test: continue
            
            # write to txt file
            print('  ', child_title)
            file.writelines(child_title+'\n')

MB_train_h36m_gt
   FIT3D-GT-ALL_TEST
   H36M-GT
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710
   3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
   3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
   H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678
   H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
   FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering
   3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
   3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
   FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST
   H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
MB_train_h36m_gt_with_canonical2
   FIT3D-GT-ALL_TEST
   H36M-GT
MB_train_h36m_gt_cam_no_factor
   3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6
   FIT3D-GT-CAM_NO_FACTOR-ALL_TEST
   3DHP-GT-CAM_NO_FACTOR-TE

In [10]:
root = f'/home/{user}/codes/MotionBERT/experiments'
config_root = 'configs/pose3d/'
checkpoint_root = 'checkpoint/pose3d/'
result_root = f'/home/{user}/codes/MotionBERT/custom_codes/evaluation/experiment_results'
if not os.path.isdir(result_root): os.makedirs(result_root)

#pbar = tqdm(glob(root+'/*.txt'))
for item in glob(root+'/*.txt'):
    model_name = os.path.basename(item).split('.')[0]
    print(f'[{model_name}]')
    config = model_name + '.yaml'
    input_args = ['--config', config_root+config, '--evaluate', checkpoint_root+model_name+'/best_epoch.bin']
    args, opts = get_opts_args(input_args, verbose=False)
    args.print_summary_table = False
    #print(args)
    try:
        model_pos, chk_filename, checkpoint = load_model(opts, args, verbose=False)
    except:
        print('  ', 'Failed to load model')
        continue
    
    with open(item, 'r') as file:
        for line in file:
            subset = line.strip()
            print(subset)
            result_path = os.path.join(result_root, model_name+'/'+subset+'.pkl')
            if os.path.exists(result_path): 
                print(f'  {subset} already exists. Skip.')
                continue
            args.subset_list = [subset]
            try:
                args.dt_file = dt_file_mapping[args.subset_list[0]]+'.pkl'
            except:
                print('  Not found in dt_file_mapping')
                continue
            print('  ', args.gt_mode, args.canonical, args.dt_file)
            train_loader_3d, test_loader, posetrack_loader_2d, instav_loader_2d, datareader = load_dataset(args)

            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            results_dict = {'total_result_dict': total_result_dict}
            savepkl(results_dict, result_path)
            
            del train_loader_3d, test_loader, posetrack_loader_2d, instav_loader_2d, datareader
            
    #pbar.set_postfix({'checkpoint': model_name})
    del model_pos, chk_filename, checkpoint, 
        


[MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z]
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN already exists. Skip.
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 already exists. Skip.
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST
  FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST already exists. Skip.
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
  H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z already exists. Skip.
[MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678]
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
  3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN already exists. Skip.
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
  3DHP-GT-CAM_NO_FACTOR-I

  0%|          | 0/245 [00:01<?, ?it/s]


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/hrai/codes/MotionBERT/lib/model/DSTformer.py", line 338, in forward
    x = self.joints_embed(x) # feature embedding - (BxF, 17, 3) -> (BxF, 17, 256)
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/torch/nn/modules/linear.py", line 114, in forward
    return F.linear(input, self.weight, self.bias)
RuntimeError: mat1 and mat2 shapes cannot be mu

In [5]:
# root = '/home/hrai/codes/MotionBERT/experiments'
# if not os.path.isdir(root): os.makedirs(root)

# sort_by_dataset = {}

# for item in glob(root+'/*.txt'):
#     model_name = os.path.basename(item).split('.')[0]
#     with open(item, 'r') as file:
#         for line in file:
#             subset = line.strip()
#             if subset not in sort_by_dataset:
#                 sort_by_dataset[subset] = [model_name]
#             else:
#                 sort_by_dataset[subset].append(model_name)